In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import sklearn
pd.set_option('display.max_rows', 2395)
pd.set_option('display.max_columns', 2395)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
import keras
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense,Dropout,Activation,LeakyReLU
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.decomposition import PCA,IncrementalPCA
from xgboost import XGBClassifier
from keras.optimizers import SGD,Adam,Adamax,Nadam
from keras.regularizers import l2
from sklearn.manifold import TSNE
import os


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
path = '/content/drive/My Drive/Colab Notebooks/DataSet'
for i,j,filename in os.walk(path):
  print(filename)

['hdfc.ipynb', 'Sample_submission.csv', 'Test.csv', 'Train.csv']
['hdfc-checkpoint.ipynb']


In [0]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DataSet/Train.csv')
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DataSet/Test.csv')

In [0]:
more_then_10 = list(df_train.columns[round(df_train.isnull().sum()/len(df_train.index),2)*100>10])
df_train = df_train.drop(more_then_10,axis =1)
df_test = df_test.drop(more_then_10,axis =1)

In [0]:
def getDuplicateColumns(df):
    duplicateColumnNames = set()
    maincol = set()
    for x in range(df.shape[1]):
        col = df.iloc[:, x]
        for y in range(x + 1, df.shape[1]):
            otherCol = df.iloc[:, y]
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y]) 
#                 maincol.add(df.columns.values[x]) 
    return list(duplicateColumnNames)

In [0]:
# duplicateColumnNames = getDuplicateColumns(df_train)
# print('Duplicate Columns are as follows')
# for col in duplicateColumnNames:
#     print('Column name : ', col)

In [0]:
# Duplicate
df_train = df_train.drop(['Col1572' ,'Col2164' 
                ,'Col493'
               ,'Col2033',
               'Col2040','Col2130'],axis=1)
df_test = df_test.drop(['Col1572' ,'Col2164'  
                    ,'Col493'
               , 'Col2033',
               'Col2040','Col2130'],axis=1)

df_train = df_train.drop(['Col489','Col1310','Col1435','Col1580'],axis='columns')
df_test = df_test.drop(['Col489','Col1310','Col1435','Col1580'],axis='columns')

In [0]:
# Duplicates
df_train = df_train.drop(['Col1302','Col1393','Col1401','Col2213','Col2074',
                          'Col1353','Col2220','Col2032','Col2131','Col1483','Col2221','Col1482'],axis=1)
df_test = df_test.drop(['Col1302','Col1393','Col1401','Col2213','Col2074',
                        'Col1353','Col2220','Col2032','Col2131','Col1483','Col2221','Col1482'],axis=1)

In [0]:
# No variance explained
df_train[['Col1797','Col1840','Col1250','Col1251','Col2024','Col1704','Col1066','Col1889','Col1203','Col1110','Col1159','Col1841','Col1705','Col1796','Col1980','Col1295','Col1111','Col1748','Col1932','Col1933']].fillna(0).astype(bool).sum(axis=0)
df_train = df_train.drop(['Col1797','Col1840','Col1250','Col1251','Col2024','Col1704','Col1066','Col1889','Col1203','Col1110',
                          'Col1159','Col1841','Col1705','Col1796','Col1980','Col1295','Col1111','Col1748','Col1932','Col1933'],axis=1)
df_test = df_test.drop(['Col1797','Col1840','Col1250','Col1251','Col2024','Col1704','Col1066','Col1889','Col1203','Col1110',
                          'Col1159','Col1841','Col1705','Col1796','Col1980','Col1295','Col1111','Col1748','Col1932','Col1933'],axis=1)

df_train = df_train.drop(['Col1749','Col1981','Col2025','Col1202','Col1888','Col1158','Col230','Col242','Col427','Col623','Col439','Col635'],axis='columns')
df_test = df_test.drop(['Col1749','Col1981','Col2025','Col1202','Col1888','Col1158','Col230','Col242','Col427','Col623','Col439','Col635'],axis='columns')

In [0]:
df_train.shape

(17521, 1872)

In [0]:
df_train[['Col1906' ,'Col1352'  ,'Col137' ,'Col510' ,'Col1311' ,
          'Col292' ,'Col1344'  ,'Col313' ,'Col1815' ,'Col1525' ,'Col1442' ,
          'Col1615' ,'Col2352' ,'Col2344' ,'Col2262' ,
          'Col2345' ,'Col1490'  ,'Col1533' ,'Col1176' ,'Col296' ,'Col1812' ,
          'Col2303'  ,'Col2263' ,'Col1622' ,'Col639' ,'Col2254' ,'Col2311' ,
          'Col1813' ,'Col2082' ,'Col2302' ,'Col2165' ,'Col2212' ,'Col1443' ,'Col2173' ,
          'Col2172' ,'Col660' ,'Col1614' ,'Col1392' 
          ,'Col1814' ,'Col1085' ,'Col2123' ,'Col2075' ,'Col1434' ,'Col1345' ,'Col1400' ,'Col1623' ,
          'Col2353' ,'Col2041' ,'Col1491' ,'Col1084' ,'Col1524' ,'Col1082' ,'Col1303' ,'Col1573' ,
          'Col2122' ,'Col1532' ,'Col1083' ,'Col2255' ,'Col2310' ,'Col2083' ,'Col1581' ,'Col643']].fillna(0).astype(bool).sum(axis=0)


Col1906     1099
Col1352    12266
Col137       381
Col510     13420
Col1311    12181
Col292     13522
Col1344    12214
Col313     13403
Col1815      180
Col1525    10402
Col1442    12128
Col1615     7813
Col2352    11524
Col2344     9948
Col2262    11868
Col2345     7765
Col1490    13223
Col1533    11573
Col1176     1134
Col296     13513
Col1812     2185
Col2303     8669
Col2263    11551
Col1622    11610
Col639     12528
Col2254    11118
Col2311    10574
Col1813     1120
Col2082    12210
Col2302    12783
Col2165    11567
Col2212    13019
Col1443    11881
Col2173    11863
Col2172    12067
Col660     12418
Col1614    10113
Col1392    13254
Col1814     1099
Col1085      185
Col2123    11577
Col2075    11965
Col1434    11923
Col1345    11983
Col1400    13254
Col1623    10614
Col2353    10576
Col2041    12167
Col1491    11697
Col1084     1134
Col1524    11220
Col1082     2577
Col1303    11676
Col1573     8694
Col2122    13089
Col1532    11941
Col1083     1138
Col2255    10365
Col2310    129

In [0]:
df_train[['Col1906' ,'Col1352'  ,'Col137' ,'Col510' ,'Col1311' ,
          'Col292' ,'Col1344'  ,'Col313' ,'Col1815' ,'Col1525' ,'Col1442' ,
          'Col1615' ,'Col2352' ,'Col2344' ,'Col2262'  ,
          'Col2345' ,'Col1490'  ,'Col1533' ,'Col1176' ,'Col296' ,'Col1812' ,
          'Col2303'  ,'Col2263' ,'Col1622' ,'Col639' ,'Col2254' ,'Col2311' ,
          'Col1813' ,'Col2082' ,'Col2302' ,'Col2165' ,'Col2212' ,'Col1443' ,'Col2173' ,
          'Col2172'  ,'Col660' ,'Col1614' ,'Col1392' 
          ,'Col1814' ,'Col1085' ,'Col2123' ,'Col2075' ,'Col1434' ,'Col1345' ,'Col1400' ,'Col1623' ,
          'Col2353' ,'Col2041' ,'Col1491' ,'Col1084' ,'Col1524' ,'Col1082' ,'Col1303' ,'Col1573' ,
          'Col2122' ,'Col1532' ,'Col1083' ,'Col2255' ,'Col2310' ,'Col2083' ,'Col1581' ,'Col643']].head(5)

,Col1906,Col1352,Col137,Col510,Col1311,Col292,Col1344,Col313,Col1815,Col1525,Col1442,Col1615,Col2352,Col2344,Col2262,Col2345,Col1490,Col1533,Col1176,Col296,Col1812,Col2303,Col2263,Col1622,Col639,Col2254,Col2311,Col1813,Col2082,Col2302,Col2165,Col2212,Col1443,Col2173,Col2172,Col660,Col1614,Col1392,Col1814,Col1085,Col2123,Col2075,Col1434,Col1345,Col1400,Col1623,Col2353,Col2041,Col1491,Col1084,Col1524,Col1082,Col1303,Col1573,Col2122,Col1532,Col1083,Col2255,Col2310,Col2083,Col1581,Col643
0,0,1,0,2.957170,2,91.5888,2,9.883509,0,0,1,0,0,0,1,0,2,0,0,83.4713,0,1,0,0,98.5294,0,1,0,1,1,1,2,0,0,1,5.485022,0,2,0,0,2,1,1,1,3,0,0,2,2,0,0,0,3,1,2,1,0,0,1,2,1,92.2608
1,0,0,2,0.000000,0,0.0000,0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0.0000,0,0,0,0,0.0000,0,0,0,0,0,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000
2,0,2,0,0.002544,0,3.4392,1,0.002040,0,0,2,0,2,1,2,0,2,0,0,0.0670,1,0,0,2,4.1935,1,0,0,2,1,1,1,0,0,2,0.002936,1,2,0,0,0,1,1,1,2,0,0,0,0,0,1,1,0,0,2,2,0,0,2,0,0,0.0868
3,0,2,0,0.037649,0,4.4160,3,0.031270,0,0,2,0,2,2,2,0,3,0,0,0.4642,0,0,0,2,4.8137,3,0,0,2,3,0,3,0,0,2,0.063382,2,3,0,0,0,0,3,0,3,0,0,0,0,0,3,0,0,0,3,2,0,0,3,0,0,0.5839
4,1,0,8,NaN,0,0.0000,0,NaN,2,0,0,0,0,0,0,0,0,0,1,0.0000,0,0,0,0,0.0000,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0000


In [0]:
# num_col = ['Col3','Col4','Col5','Col6','Col7','Col8','Col9','Col10','Col11','Col12','Col25','Col26','Col168','Col169','Col170',
#                'Col181','Col182','Col186','Col187','Col188','Col189','Col190','Col191','Col192','Col193','Col194','Col195','Col196',
#                'Col197','Col198','Col199','Col200','Col201','Col202','Col203','Col204','Col205','Col206','Col207','Col378','Col379',
#                'Col383','Col384','Col385','Col386','Col387','Col388','Col389','Col390','Col391','Col392','Col393','Col394','Col395',
#                'Col396','Col397','Col398','Col399','Col400','Col401','Col402','Col403','Col404','Col575','Col576','Col579','Col580',
#                'Col581','Col582','Col583','Col584','Col585','Col586','Col587','Col588','Col589','Col590','Col591','Col592','Col593',
#                'Col594','Col595','Col596','Col597','Col598','Col599','Col600']
# def drop_outliers(x):
#     list = []
#     for col in num_col:
#         Q1 = x[col].quantile(.25)
#         Q3 = x[col].quantile(.99)
#         IQR = Q3-Q1
#         x =  x[(x[col] >= (Q1-(1.5*IQR))) & (x[col] <= (Q3+(1.5*IQR)))] 
#     return x     
# print(df_train.shape)
# df_train = drop_outliers(df_train)
# print(df_train.shape)

In [0]:
print('Train_Shape:{0},Test Shape{1}'.format(df_train.shape,df_test.shape))

Train_Shape:(17521, 1872),Test Shape(20442, 1871)


In [0]:
df_train[list(round((df_train.isnull().sum()/len(df_train))*100,2)[round((df_train.isnull().sum()/len(df_train))*100,2)>0.00].index)].head()

,Col13,Col14,Col15,Col16,Col17,Col21,Col22,Col23,Col24,Col27,Col42,Col43,Col92,Col106,Col162,Col163,Col164,Col165,Col166,Col167,Col175,Col176,Col177,Col178,Col179,Col180,Col208,Col213,Col214,Col215,Col216,Col217,Col218,Col219,Col220,Col221,Col222,Col223,Col224,Col225,Col226,Col227,Col228,Col229,Col231,Col232,Col233,Col234,Col235,Col236,Col237,Col238,Col239,Col240,Col241,Col243,Col244,Col245,Col246,Col247,Col292,Col293,Col294,Col295,Col296,Col297,Col298,Col299,Col300,Col301,Col302,Col303,Col304,Col305,Col306,Col307,Col308,Col309,Col310,Col311,Col312,Col313,Col314,Col315,Col316,Col317,Col318,Col319,Col320,Col321,Col322,Col323,Col324,Col325,Col326,Col327,Col328,Col329,Col405,Col410,Col411,Col412,Col413,Col414,Col415,Col416,Col417,Col418,Col419,Col420,Col421,Col422,Col423,Col424,Col425,Col426,Col428,Col429,Col430,Col431,Col432,Col433,Col434,Col435,Col436,Col437,Col438,Col440,Col441,Col442,Col443,Col444,Col490,Col491,Col492,Col494,Col495,Col496,Col497,Col498,Col499,Col500,Col501,Col502,Col503,Col504,Col505,Col506,Col507,Col508,Col509,Col510,Col511,Col512,Col513,Col514,Col515,Col516,Col517,Col518,Col519,Col520,Col521,Col522,Col523,Col524,Col525,Col526,Col601,Col606,Col607,Col608,Col609,Col610,Col611,Col612,Col613,Col614,Col615,Col616,Col617,Col618,Col619,Col620,Col621,Col622,Col624,Col625,Col626,Col627,Col628,Col629,Col630,Col631,Col632,Col633,Col634,Col636,Col637,Col638,Col639,Col640,Col641,Col642,Col643,Col644,Col645,Col646,Col647,Col648,Col649,Col650,Col651,Col652,Col653,Col654,Col655,Col656,Col657,Col658,Col659,Col660,Col661,Col662,Col663,Col664,Col665,Col666,Col667,Col668,Col669,Col670,Col671,Col672,Col673,Col674,Col675,Col676,Col680,Col681,Col682,Col683,Col691,Col693,Col703,Col705,Col771,Col772,Col780,Col782,Col792,Col794,Col860,Col861,Col862,Col863,Col866,Col867,Col870,Col871,Col874,Col875,Col878,Col879,Col880,Col881,Col882,Col883,Col884,Col885,Col916,Col921,Col926,Col931,Col932,Col933,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,9.293496,8.570479,10.063117,10.271758,0.548538,1.486524,1.728324,0.993441,1.297050,0.260428,6.07,6.07,17.980863,17.980863,2.565795,0.221124,2.565795,0.221124,2.636174,0.341804,0.983505,143.278744,0.983505,42.869348,1.106012,33.961214,0.9346,0.0000,91.5888,0.4673,0.0000,0.0000,1.4019,1.8692,91.5888,92.0561,4.0689,0.0000,83.4713,0.2747,0.0000,0.0000,0.0000,4.1690,11.8813,83.4713,83.7461,0.481788,0.000000,9.883509,0.032530,0.000000,0.0,0.000000,0.493640,1.40682,9.883509,9.916040,0.582386,0.582423,91.5888,0.0000,0.0000,0.0000,83.4713,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,83.4713,0.0000,0.0000,0.0,83.4713,83.4713,0.0000,0.0,0.0000,9.883509,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,9.883509,0.000000,0.0,0.0,9.883509,9.883509,0.000000,0.0,0.000000,0.9346,0.0000,91.5888,0.4673,0.0000,0.0000,1.4019,1.8692,91.5888,92.0561,4.0689,0.0000,83.4713,0.2747,0.0000,0.0000,0.0000,4.1690,11.8813,83.4713,83.7461,0.144152,0.000000,2.957170,0.009733,0.000000,0.0,0.000000,0.147698,0.420924,2.957170,2.966903,0.582386,0.582423,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,83.4713,0.0000,0.0000,0.0,83.4713,83.4713,0.0000,0.0,0.0000,2.957170,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,2.957170,0.000000,0.0,0.0,2.957170,2.957170,0.000000,0.0,0.000000,1.4706,0.0000,98.5294,0.0000,0.0000,0.0000,0.0000,1.4706,98.5294,98.5294,7.7392,0.0000,92.2608,0.0000,0.000,0.0000,0.0000,0.0000,7.7392,92.2608,92.2608,0.460104,0.000000,5.485022,0.000000,0.000000,0.0,0.000000,0.000000,0.460104,5.485022,5.485022,98.5294,0.0000,0.0000,0.0000,92.2608,0.0,0.0,0.000,0.0000,0.000,0.000,0.0000,92.2608,0.0000,0.0000,0.0,92.2608,92.2608,0.0000,0.0,0.0000,5.485022,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,5.485022,0.000000,0.0,0.0,5.485022,5.485022,0.000000,0.0,0.000000,-0.255842,-0.382431,-0.255842,-0.382431,-0.111062,0.212767,-0.076573,0.146766,-0.088258,-0.088258,-0.111062,0.212767,-0.076573,0.146766,-0.088258,-0.088258,40.0,40.0,0.0,0.0,0.755869,0.755869,4.015786,4.015786,39.0,1.0,7.619048,1.099273,39.0,1.0,7.6

In [0]:
df_train.head()

Col1  Col2          Col3          Col4          Col5          Col6          Col7          Col8          Col9         Col10         Col11         Col12      Col13      Col14      Col15      Col16     Col17     Col21     Col22     Col23     Col24     Col25     Col26     Col27  Col29  Col30  Col31  Col32  Col33  Col34  Col35  Col36   Col42   Col43  Col44  Col45  Col46  Col47  Col48  Col49  Col50  Col51  Col52  Col53  Col54  Col55  Col56  Col57  Col58  Col59  Col60  Col61  Col62  Col63  Col64  Col65  Col66  Col67  Col68  Col69  Col70  Col71  Col72  Col73  Col74  Col75  Col76  Col77  Col78  Col79  Col80  Col81  Col82  Col83  Col84  Col85  Col86  Col87  Col88  Col89  Col90  Col91      Col92     Col106  Col120  Col121  Col122  Col123  Col124  Col125  Col126  Col127  Col128  Col129  Col130  Col131  Col132  Col133  Col134  Col135  Col136  Col137  Col160  Col161    Col162    Col163    Col164    Col165    Col166    Col167        Col168        Col169        Col170    Col175      Col176  \
0    RIGHGSN     1  1.086711e+04  8.648345e+03  1.576618e+04  2.890466e+04  5.273655e+03  2.346153e+04  1.285597e+04  2.871509e+04  9.114586e+03  3.043078e+04   9.293496   8.570479  10.063117  10.271758  0.548538  1.486524  1.728324  0.993441  1.297050  0.137398  0.115987  0.260428      1      1      1      1      1      1      0      6    6.07    6.07      2      3      2      3      2      2      2      2      1      3      1      3      1      2      1      2      1      1      1      1      1      1      1      1      3      1      3      1      3      1      3      1      2      1      2      1      2      1      2      1      1      0      1      0      1      0      1      0  17.980863  17.980863       4       1       4       1       3       1       1       1       3       1       3       1       2       1       0       1       0       0      14      14  2.565795  0.221124  2.565795  0.221124  2.636174  0.341804  6.247142e+05  6.247142e+05  1.752412e+05  0.983505  143.278744   
1  RIMHI5ZGP     0  2.584312e+05  2.179633e+05  1.866287e+05  1.866287e+05  2.154519e+05  1.923615e+05  1.726538e+05  1.666974e+05  2.013518e+05  1.859404e+05  12.462385  12.280493  12.167132  12.136876  1.167898  0.792123  0.934556  0.893203  0.966620  0.753892  0.808295  1.414214      1      1      3      4      4      0      4      5  116.65  116.65      3      0      2      0      2      0      1      0      3      0      2      0      2      0      1      0      2      0      2      0      2      0      1      0      2      0      2      0      2      0      2      0      2      0      2      0      2      0      2      0      1      0      1      0      1      0      1      0   0.957537   0.957537       1       0       1       0       1       0       1       0       1       0       1       0       1       0       1       0       2       2       2       2  7.575385  0.511835  7.575385  0.511835  9.210323  0.667423  5.282476e+06  5.282476e+06  5.182476e+06  0.893896   45.899741   
2  RIHOG1SZU     0  1.459355e+07  1.260060e+07  1.100315e+07  1.010315e+07  1.879882e+07  1.514035e+07  5.320778e+06  6.906324e+06  9.368338e+06  1.113330e+07  16.496090  16.749305  16.532874  16.128357  1.145182  0.422264  0.498347  0.683582  0.735339  0.808115  0.776910  1.414214      3      3      4     12      4      0      2      4   33.28   33.28      2      1      2      0      1      0      1      0      2      1      2      0      1      0      1      0      1      1      1      0      1      0      1      0      2      1      2      1      1      0      1      0      2      1      2      1      1      0      1      0      1      1      1      1      1      0      1      0   0.482401   0.482401       1       0       1       0       1       0       0       0       1       0       1       0       1       0       0       0       0       0       0       0  3.334681  0.184931  3.334681  0.184931  3.219582  0.223574  2.226136e+08  2.226136e+08  1.701652e+08  0.861632   32.889272   
3  RIHOL

In [0]:
df_train.dtypes[df_train.dtypes=='object']

Col1    object
dtype: object

In [0]:
train_y = df_train.iloc[:,1]
train_X = df_train.drop('Col2',axis='columns')
test_X = df_test

In [0]:
train_X = train_X.drop('Col1',axis=1)
test_X = test_X.drop('Col1',axis=1)

In [0]:
miss_val_col = list(train_X.isnull().sum()[train_X.isnull().sum()>0].index)
def impute_mean(df):
  for col in miss_val_col:
    df.loc[df[col].isnull(),col] = df[col].mean()
impute_mean(train_X)
impute_mean(test_X)

In [0]:
train_X[miss_val_col].head()

,Col13,Col14,Col15,Col16,Col17,Col21,Col22,Col23,Col24,Col27,Col42,Col43,Col92,Col106,Col162,Col163,Col164,Col165,Col166,Col167,Col175,Col176,Col177,Col178,Col179,Col180,Col208,Col213,Col214,Col215,Col216,Col217,Col218,Col219,Col220,Col221,Col222,Col223,Col224,Col225,Col226,Col227,Col228,Col229,Col231,Col232,Col233,Col234,Col235,Col236,Col237,Col238,Col239,Col240,Col241,Col243,Col244,Col245,Col246,Col247,Col292,Col293,Col294,Col295,Col296,Col297,Col298,Col299,Col300,Col301,Col302,Col303,Col304,Col305,Col306,Col307,Col308,Col309,Col310,Col311,Col312,Col313,Col314,Col315,Col316,Col317,Col318,Col319,Col320,Col321,Col322,Col323,Col324,Col325,Col326,Col327,Col328,Col329,Col405,Col410,Col411,Col412,Col413,Col414,Col415,Col416,Col417,Col418,Col419,Col420,Col421,Col422,Col423,Col424,Col425,Col426,Col428,Col429,Col430,Col431,Col432,Col433,Col434,Col435,Col436,Col437,Col438,Col440,Col441,Col442,Col443,Col444,Col490,Col491,Col492,Col494,Col495,Col496,Col497,Col498,Col499,Col500,Col501,Col502,Col503,Col504,Col505,Col506,Col507,Col508,Col509,Col510,Col511,Col512,Col513,Col514,Col515,Col516,Col517,Col518,Col519,Col520,Col521,Col522,Col523,Col524,Col525,Col526,Col601,Col606,Col607,Col608,Col609,Col610,Col611,Col612,Col613,Col614,Col615,Col616,Col617,Col618,Col619,Col620,Col621,Col622,Col624,Col625,Col626,Col627,Col628,Col629,Col630,Col631,Col632,Col633,Col634,Col636,Col637,Col638,Col639,Col640,Col641,Col642,Col643,Col644,Col645,Col646,Col647,Col648,Col649,Col650,Col651,Col652,Col653,Col654,Col655,Col656,Col657,Col658,Col659,Col660,Col661,Col662,Col663,Col664,Col665,Col666,Col667,Col668,Col669,Col670,Col671,Col672,Col673,Col674,Col675,Col676,Col680,Col681,Col682,Col683,Col691,Col693,Col703,Col705,Col771,Col772,Col780,Col782,Col792,Col794,Col860,Col861,Col862,Col863,Col866,Col867,Col870,Col871,Col874,Col875,Col878,Col879,Col880,Col881,Col882,Col883,Col884,Col885,Col916,Col921,Col926,Col931,Col932,Col933,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,9.293496,8.570479,10.063117,10.271758,0.548538,1.486524,1.728324,0.993441,1.297050,0.260428,6.070000,6.070000,17.980863,17.980863,2.565795,0.221124,2.565795,0.221124,2.636174,0.341804,0.983505,143.278744,0.983505,42.869348,1.106012,33.961214,0.9346,0.0000,91.5888,0.4673,0.0000,0.0000,1.4019,1.8692,91.5888,92.0561,4.0689,0.0000,83.4713,0.2747,0.0000,0.0000,0.0000,4.1690,11.8813,83.4713,83.7461,0.481788,0.000000,9.883509,0.032530,0.000000,0.000000,0.000000,0.493640,1.406820,9.883509,9.916040,0.582386,0.582423,91.5888,0.0000,0.0000,0.0000,83.4713,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,83.4713,0.0000,0.0000,0.0,83.4713,83.4713,0.0000,0.0,0.0000,9.883509,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.883509,0.000000,0.000000,0.000000,9.883509,9.883509,0.000000,0.00000,0.000000,0.9346,0.0000,91.5888,0.4673,0.0000,0.0000,1.4019,1.8692,91.5888,92.0561,4.0689,0.0000,83.4713,0.2747,0.0000,0.0000,0.0000,4.1690,11.8813,83.4713,83.7461,0.144152,0.000000,2.957170,0.009733,0.000000,0.000000,0.000000,0.147698,0.420924,2.957170,2.966903,0.582386,0.582423,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,83.4713,0.0000,0.0000,0.0,83.4713,83.4713,0.0000,0.0,0.0000,2.957170,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.957170,0.000000,0.000000,0.000000,2.957170,2.957170,0.000000,0.000000,0.000000,1.4706,0.0000,98.5294,0.0000,0.0000,0.0000,0.0000,1.4706,98.5294,98.5294,7.7392,0.0000,92.2608,0.0000,0.000,0.0000,0.0000,0.0000,7.7392,92.2608,92.2608,0.460104,0.000000,5.485022,0.000000,0.000000,0.000000,0.000000,0.000000,0.460104,5.485022,5.485022,98.5294,0.0000,0.0000,0.0000,92.2608,0.0,0.0,0.000,0.0000,0.000,0.000,0.0000,92.2608,0.0000,0.0000,0.0,92.2608,92.2608,0.0000,0.0,0.0000,5.485022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.485022,0.000000,0.000000,0.00000,5.485022,5.485022,0.000000,0.000000,0.000000,-0.255842,-0.382431,-0.255842,-0.382431,-0.111062,0.212767,-0.076573,0.146766,-0.088258,-0.088258,-0.111062,0.212767,-0.076573,0.146766,-

In [0]:
print(train_X.isnull().sum()[train_X.isnull().sum()>0])
print(test_X.isnull().sum()[test_X.isnull().sum()>0])

Series([], dtype: int64)
Series([], dtype: int64)


In [0]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
my_imputer = IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=np.NaN, n_nearest_features=None,
                 random_state=0, sample_posterior=False, tol=0.001,
                 verbose=0)
# train_X = my_imputer.fit_transform(train_X)

In [0]:
# test_X= my_imputer.transform(test_X)

In [0]:
print('Train_Shape:{0},Test Shape{1}'.format(train_X.shape,test_X.shape))

Train_Shape:(17521, 1870),Test Shape(20442, 1870)


In [0]:
# train_X = train_X.drop(l,axis=1)
# test_X = test_X.drop(l,axis=1)

In [0]:
scale_col = list(train_X.columns)
scale = StandardScaler()
train_X[scale_col] = scale.fit_transform(train_X[scale_col])
test_X[scale_col] = scale.transform(test_X[scale_col])

In [0]:
pca = PCA(.98)
train_X_PCA = pca.fit_transform(train_X)
test_X_PCA = pca.transform(test_X)

In [0]:
print('Train_Shape:{0},Test Shape{1}'.format(train_X_PCA.shape,test_X_PCA.shape))

Train_Shape:(17521, 336),Test Shape(20442, 336)


In [0]:
smote = SMOTE()
train_X_PCA,train_y = smote.fit_resample(train_X_PCA,train_y)

In [0]:
train_y = keras.utils.to_categorical(train_y)
train_y.shape

(31520, 2)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [0]:
  model = Sequential()
  model.add(Dense(512,input_dim =train_X_PCA.shape[1] ,kernel_initializer='normal',activation='relu'))
  model.add(Dropout(.30))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dropout(.30))
  model.add(Dense(128, kernel_initializer='normal',activation='relu'))
  model.add(Dropout(.30))
  model.add(Dense(64, kernel_initializer='normal',activation='relu'))
  model.add(Dropout(.20))
  model.add(Dense(32, kernel_initializer='normal',activation='relu'))
  model.add(Dropout(.20))
  model.add(Dense(32, kernel_initializer='normal',activation='relu'))
  model.add(Dropout(.20))
  model.add(Dense(16, kernel_initializer='normal',activation='relu'))
  model.add(Dropout(.20))
  model.add(Dense(2, kernel_initializer='normal',activation='sigmoid'))


In [0]:
# epochs = 30
# lrate = 0.01
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.4, decay=decay, nesterov=False)
# adam = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 512)               172544    
_________________________________________________________________
dropout_30 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_31 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_32 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 64)               

In [0]:
model.fit(train_X_PCA,train_y,batch_size=128,epochs=200,shuffle=True,verbose=1,validation_split=0.2)

Train on 25216 samples, validate on 6304 samples
Epoch 1/200
25216/25216 [==============================] - 3s 108us/step - loss: 0.5823 - acc: 0.6711 - val_loss: 0.7753 - val_acc: 0.6748
Epoch 2/200
25216/25216 [==============================] - 2s 64us/step - loss: 0.4861 - acc: 0.7684 - val_loss: 0.6425 - val_acc: 0.8885
Epoch 3/200
25216/25216 [==============================] - 2s 64us/step - loss: 0.4204 - acc: 0.8122 - val_loss: 0.5999 - val_acc: 0.8361
Epoch 4/200
25216/25216 [==============================] - 2s 64us/step - loss: 0.3739 - acc: 0.8418 - val_loss: 0.5222 - val_acc: 0.8580
Epoch 5/200
25216/25216 [==============================] - 2s 64us/step - loss: 0.3330 - acc: 0.8617 - val_loss: 0.4606 - val_acc: 0.8622
Epoch 6/200
25216/25216 [==============================] - 2s 64us/step - loss: 0.3008 - acc: 0.8801 - val_loss: 0.3943 - val_acc: 0.8958
Epoch 7/200
25216/25216 [==============================] - 2s 64us/step - loss: 0.2754 - acc: 0.8914 - val_loss: 0.3685 - 

In [0]:
y_test = model.predict(test_X_PCA)

In [0]:
y_classes = [np.argmax(y, axis=None, out=None) for y in y_test]

In [0]:
df  = pd.DataFrame({'Col1':df_test.Col1,'Col2':y_classes})
df.set_index('Col1', inplace=True)

In [0]:
df.to_csv('df.csv')

In [0]:
  from google.colab import files
  files.download('df.csv')

In [0]:

xgb_model = XGBClassifier(learning_rate=0.02,objective='binary:logistic',silent=False,nthread=1)

parameters = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'n_estimators': range(100, 1500, 400)
        }


clf = RandomizedSearchCV(xgb_model, 
                         parameters, n_jobs=5, 
                         cv=5,
                         scoring='roc_auc',
                         verbose=2, 
                       refit=True)

In [0]:
clf.fit(train_X_PCA,train_y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed: 73.2min
[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed: 129.1min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.02, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=1,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           r...
                                           seed=None, silent=False, subsample=1,
                                           verbosity=1),
                   iid='warn', n_iter=10

In [0]:
clf.best_params_

{'colsample_bytree': 1.0,
 'gamma': 1,
 'max_depth': 5,
 'min_child_weight': 10,
 'n_estimators': 1300,
 'subsample': 1.0}

In [0]:
clf.best_score_

0.9390192392550697

In [0]:
# train_y = [np.argmax(y, axis=None, out=None) for y in train_y]
xgb_model_final = XGBClassifier(learning_rate=0.02,
                                colsample_bytree= 1.0,
                                n_estimators=1300,
                                objective='binary:logistic',
                                silent=False,
                                nthread=1,
                                
                                gamma=1,
                                max_depth=5,
                                min_child_weight=10,
                                subsample=1.0)
xgb_model_final.fit(train_X_PCA,train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=1,
              learning_rate=0.02, max_delta_step=0, max_depth=5,
              min_child_weight=10, missing=None, n_estimators=1300, n_jobs=1,
              nthread=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=1.0, verbosity=1)

In [0]:
y_classes = xgb_model_final.predict(test_X_PCA)

In [0]:
# train_y = [np.argmax(y, axis=None, out=None) for y in train_y]
pd.DataFrame({'Col1':df_test.Col1,'Col2':y_classes}).to_csv('df.csv')
from google.colab import files
files.download('df.csv')

In [0]:
!pip install lightgbm

In [0]:
train_y = [np.argmax(y, axis=None, out=None) for y in train_y]
import lightgbm as lgb
d_train = lgb.Dataset(train_X_PCA, label=train_y)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

In [0]:
test_y_prob = clf.predict(test_X_PCA)

In [0]:
test_y_class = np.where(test_y_prob > 0.2, 1, 0) 

In [0]:
pd.DataFrame({'Col1':df_test.Col1,'Col2':test_y_class}).to_csv('df.csv')
from google.colab import files
files.download('df.csv')

In [0]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(solver='adam', activation='relu',alpha=1e-4,hidden_layer_sizes=(50,50,50), random_state=1,max_iter=20,verbose=10,learning_rate_init=.1)
model.fit(train_X_PCA, train_y)

Iteration 1, loss = inf
Iteration 2, loss = inf
Iteration 3, loss = 0.63890465
Iteration 4, loss = 0.58951492
Iteration 5, loss = 0.55828118
Iteration 6, loss = 0.53638769
Iteration 7, loss = inf
Iteration 8, loss = 0.50612862
Iteration 9, loss = 0.49230570
Iteration 10, loss = inf
Iteration 11, loss = 0.47717720
Iteration 12, loss = 0.48088920
Iteration 13, loss = 0.50735840
Iteration 14, loss = 0.48721073
Iteration 15, loss = 0.45820525
Iteration 16, loss = 0.46801813
Iteration 17, loss = 0.45899090
Iteration 18, loss = 0.45778798
Iteration 19, loss = 0.44286772
Iteration 20, loss = inf


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 50, 50), learning_rate='constant',
              learning_rate_init=0.1, max_iter=20, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=10, warm_start=False)

In [0]:
mlp = model.predict(test_X_PCA)

In [0]:
train_y

array([1, 0, 0, ..., 1, 1, 1])

In [0]:
# y_classes = [np.argmax(y, axis=None, out=None) for y in mlp]
df  = pd.DataFrame({'Col1':df_test.Col1,'Col2':mlp})
df.set_index('Col1', inplace=True)
df.to_csv('df.csv')

In [0]:
from google.colab import files
files.download('df.csv')

In [0]:
from sklearn.neural_network import MLPClassifier
parameters = {'solver': ['lbfgs','adam'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}

In [0]:
clf = RandomizedSearchCV(MLPClassifier(), parameters, n_jobs=-1,verbose=1)

In [0]:
clf.fit(train_X_PCA, train_y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 17.8min finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_iter=200, momentum=0.9,
                                           n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5...
                   param_distributions={'alpha': array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07, 1.e-08,
       1.e-09]),
                                        'hidden_layer_sizes': array([10, 11, 12, 13, 14

In [0]:
clf.best_params_

In [0]:
train_X_PCA.shape

In [0]:
new = MLPClassifier(alpha=1e-05, 
                    max_iter=1800,
                    random_state=5,
                    solver='adam',
                    hidden_layer_sizes=(14),
                    early_stopping=True,
                   batch_size=32)

In [0]:
new.fit(train_X_PCA,train_y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size=32, beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=14, learning_rate='constant',
              learning_rate_init=0.001, max_iter=1800, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=5, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
mlp = new.predict(test_X_PCA)

In [0]:
# y_classes = [np.argmax(y, axis=None, out=None) for y in mlp]
df  = pd.DataFrame({'Col1':df_test.Col1,'Col2':mlp})
df.set_index('Col1', inplace=True)
df.to_csv('df.csv')

In [0]:
from google.colab import files
files.download('df.csv')

In [0]:
np.unique(mlp)

array([0, 1])

In [0]:
mlp

array([0, 0, 0, ..., 0, 0, 0])